In [1]:
import Post_Donors_PreProcess

In [2]:
import GPy
import GPyOpt
import numpy as np

In [3]:
import lightgbm as lgbm
from sklearn.metrics import roc_auc_score

In [4]:
#Post_Donor_PrePro : Preprocessing function that pulls from the original data
#Diet : Pull from a previously saved preprocessed csv, as to only do scaling and text processing

In [ ]:
#X, y, df_cols, le_dict = Post_Donors_PreProcess.Post_Donor_PrePro(
#                            Tf_Features=100, Sample=1, 
#                            One_Hot=False, Standard_Scale=True)

In [ ]:
X, y, df_cols, le_dict = Post_Donors_PreProcess.Diet_Prepro(
                            Tf_Features=100, 
                            One_Hot=False, 
                            Standard_Scale=True)

Encoding
Scaling
Text Processing


In [ ]:
lgbm_train_data = lgbm.Dataset(data=X#.drop('School Percentage Free Lunch',axis=1)
                               ,label=y)

In [ ]:
y.value_counts(normalize=True)

## Default

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [ ]:
model = lgbm.LGBMClassifier()
model.fit(X_train,y_train)

In [ ]:
model.score(X_test,y_test)

## Random Search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
model = lgbm.LGBMClassifier(
    boosting_type = 'dart',
    application= 'binary',
    learning_rate= 0.01,
    scale_pos_weight= 2,
    drop_rate= 0.2,)

In [ ]:
params_rand2 = {'n_estimators': [x for x in range(50,200,20)],
             'feature_fraction': [x / 1000.0 for x in range(820,900,20)],
             'bagging_fraction': [x / 1000.0 for x in range(820,900,20)],
             'lambda_l2' : [x / 100.0 for x in range(0,310,10)],
             'boosting_type': ['dart','gbdt']}

In [ ]:
rs = RandomizedSearchCV(
    model, 
    params_rand2, 
    n_iter=5,
    scoring='roc_auc',
    cv=5)

rs.fit(X,y)

In [ ]:
rs.best_score_

In [ ]:
rs.best_params_

## Bay Search

In [ ]:
params_bay = [{'name': 'feature_fraction','type': 'continuous', 'domain': (0.5,0.9)},
              {'name': 'bagging_fraction','type': 'continuous', 'domain': (0.5,0.9)},
              {'name': 'lambda_l2',       'type': 'continuous', 'domain': (0.1,0.5)},
              {'name': 'n_estimators',      'type': 'discrete', 'domain': (100,300)}
             ]

In [ ]:
def Bay_Wrapper(x):
    """
    Model Wrapper for Bay Opt 
    Wrapper used for LGBM & Bay:
    x[0][:] = list of all params for this aq
    """
    AUC = True
    if AUC: metric = 'auc'
    else: metric = 'binary_logloss'
    
    lgbm_params = { 'feature_fraction': x[0][0],
                    'bagging_fraction': x[0][1],
                    'lambda_l2' : x[0][2],
                    'boosting': 'dart',
                   
                   
                    'n_estimators' : int(x[0][3]),
                    #'num_leaves': int(x[0][3]),
                    'max_depth': -1,
                                 #int(x[0][4]),
                   
                    'application': 'binary',
                    'learning_rate': 0.01,
                    'scale_pos_weight': 2,
                    'drop_rate': 0.2,
}

    cv_results = lgbm.cv(train_set=lgbm_train_data,
                         params=lgbm_params, 
                         nfold=5,
                         num_boost_round = 600,
                         early_stopping_rounds=50,
                         metrics=[metric])

    optimum_boost_rounds = np.argmax(cv_results[metric+'-mean'])
    print('{} : {}'.format(metric,np.max(cv_results['{}-mean'.format(metric)])))
    
    return np.max(cv_results[metric+'-mean'])

In [ ]:
bay = GPyOpt.methods.BayesianOptimization(Bay_Wrapper,
                                          params_bay,
                                          acquisition_type ='EI',   # LCB acquisition
                                          acquisition_weight = .5,  # Exploration exploitation
                                          maximize=True
                                         )
bay.run_optimization(50) #GPyOpt uses 5 points + the number listed
print('Function after 5 Aquisitions')
bay.plot_acquisition()

In [ ]:
bay.plot_convergence()

In [ ]:
bay.plot_acquisition()

In [ ]:
bay_pred = bay.x_opt
print('feature_fraction: '+str(round(bay_pred[0],5)))
print('bagging_fraction: '+str(round(bay_pred[1],5)))
print('lambda_l2: '+str(round(bay_pred[2],5)))
print('n_estimators: '+str(round(bay_pred[3],5)))

In [ ]:
lgbm_params = { 'boosting': 'dart',
                'application': 'binary',
                'learning_rate': 0.01,
                'scale_pos_weight': 2,
                'drop_rate': 0.2,
                'n_estimators': int(bay_pred[3]),
                'feature_fraction': bay_pred[0],
                'bagging_fraction': bay_pred[1],
                'lambda_l2' : bay_pred[2]}

cv_results = lgbm.cv(train_set=lgbm_train_data,
                     params=lgbm_params, 
                     nfold=5,
                     num_boost_round=600,
                     early_stopping_rounds=50,
                     #metrics=['accuracy']
                    )
#'binary_logloss-mean'
optimum_boost_rounds = np.argmax(cv_results['binary_logloss-mean'])
print('Auc : {}'.format(np.max(cv_results['binary_logloss-mean'])))